# Gradient boosted decision tree

## Что первое пришло в голову

Необычная NLP задача, по факту нужно классифицировать класс каждой буквы в слове на основании каких-то признаков.


Потенциальные признаки: векторное представление слова (циферка для каждой буквы, желательно по-умному нормализовать, потому что нормализовать по алфавиту - не очень, скорее всего нужно будет присваивать цифру в зависимости от частоты встречаемости), длина слова, число гласных/согласных, позиция слова в изначальном названии компании, внутри ли кавычек/скобочек слово.

In [1]:
import numpy as np
import pandas as pd
import random

random.seed(42)

# Длина максимального слова (для нормализации), подробнее в 'Testing.ipynb'
MAX_WORD = 57

CONSONANTS = 'бвгджзйклмнпрстфхцчшщБВГДЖЗЙКЛМНПРСТФХЦЧШЩbcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXZ'
VOWELS = 'аиоуэыеёюяАИОУЭЫЕЁЮЯaeiouyAEIOUY'
ALPHABET = 'АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюяaeiouyAEIOUYbcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXZ'

In [2]:
train_companies = open('../kontur_srs_internship_test_task/train.txt', 'r').readlines()
train_companies = list(map(lambda s: s.strip(), train_companies))

test_companies = open('../kontur_srs_internship_test_task/test.txt', 'r').readlines()
test_companies = list(map(lambda s: s.strip(), test_companies))

In [3]:
import re
               
def tokenize(company, spaces=False):
    with_spaces = list(filter(None, re.split('(\w+| )', company)))
    if(spaces):
        return with_spaces
    return [token for token in with_spaces if token != ' ']

def has_letters(string):
    return any(char in ALPHABET for char in string)

def has_not_letters(string):
    return any(char not in ALPHABET for char in string)

def count_vowels(string):
    return len([char for char in string if char in VOWELS])

def count_consonants(string):
    return len([char for char in string if char in CONSONANTS])

<font size='3'> В данной строке артефакт, подробнее в 'Analysis.ipynb'</font>

In [4]:
del train_companies[471970]

In [5]:
def extract_features(company, test=False):
    
    # Я достаю длину слова, количество гласных/согласных, положение в слове,
    # узнаю есть ли в слове что-то помимо букв, какой символ окружает слово
    # и то же самое для предыдущего и последующего слова
    
    num_of_features = 15 + MAX_WORD
    length = num_of_features * 3
    
    # 3 класса - полностью писменными, первая заглавная или полностью заглавными
    num_of_classes = 3

    size = 0
    for word in tokenize(company):
        if(has_letters(word)):
            size += 1
            
    features = np.zeros((size, length))
    labels = np.zeros(size)
    
    index = 0
    tokens = tokenize(company, spaces=True)
    for i, word in enumerate(tokens):
        if(has_letters(word)):
            
            # Генерация фич
            
            if(index == 0):
                features[index][num_of_features] = index/size
                features[index][num_of_features + 1] = len(word)/MAX_WORD
                features[index][num_of_features + 2] = count_consonants(word)/len(word)
                features[index][num_of_features + 3] = count_vowels(word)/len(word)
                features[index][num_of_features + 4] = int(has_not_letters(word))
                if(i > 0):
                    features[index][num_of_features + 5] = int(tokens[i - 1] == '"')
                    features[index][num_of_features + 6] = int(tokens[i - 1] == '(')
                    features[index][num_of_features + 7] = int(tokens[i - 1] == ' ')
                    features[index][num_of_features + 8] = int(tokens[i - 1] == ')')
                    features[index][num_of_features + 9] = int(tokens[i - 1] == '-')
                if( i + 1 < len(tokens)):
                    features[index][num_of_features + 10] = int(tokens[i + 1] == '""')
                    features[index][num_of_features + 11] = int(tokens[i + 1] == '(')
                    features[index][num_of_features + 12] = int(tokens[i + 1] == ' ')
                    features[index][num_of_features + 13] = int(tokens[i + 1] == ')')
                    features[index][num_of_features + 14] = int(tokens[i + 1] == '-')
                    
                for j, char in enumerate(list(word)):
                    features[index][num_of_features + 15 + j] = encoding[char.upper()]
                
            else:
                features[index - 1][2 * num_of_features] = index/size
                features[index - 1][2 * num_of_features + 1] = len(word)/MAX_WORD
                features[index - 1][2 * num_of_features + 2] = count_consonants(word)/len(word)
                features[index - 1][2 * num_of_features + 3] = count_vowels(word)/len(word)
                features[index - 1][2 * num_of_features + 4] = int(has_not_letters(word))
                features[index - 1][2 * num_of_features + 5] = int(tokens[i - 1] == '"')
                features[index - 1][2 * num_of_features + 6] = int(tokens[i - 1] == '(')
                features[index - 1][2 * num_of_features + 7] = int(tokens[i - 1] == ' ')
                features[index - 1][2*num_of_features + 8] = int(tokens[i - 1] == ')')
                features[index - 1][2*num_of_features + 9] = int(tokens[i - 1] == '-')
                if( i + 1 < len(tokens)):
                    features[index - 1][2 * num_of_features + 10] = int(tokens[i + 1] == '"')
                    features[index - 1][2 * num_of_features + 11] = int(tokens[i + 1] == '(')
                    features[index - 1][2 * num_of_features + 12] = int(tokens[i + 1] == ' ')
                    features[index - 1][2*num_of_features + 13] = int(tokens[i + 1] == ')')
                    features[index - 1][2*num_of_features + 14] = int(tokens[i + 1] == '-')
                
                for j, char in enumerate(list(word)):
                    features[index - 1][2 * num_of_features + 15 + j] = encoding[char.upper()]
                
                features[index][:2 * num_of_features] = features[index - 1][num_of_features:]
            
            # Присваивание лейбла (слова, которые записаны
            # не строчными и не заглавными полностью отношу к словам 
            # к словам с заглавной буквы
            # 0 - полностью строчные
            # 1 - с заглавной буквы
            # 2 - полностью заглавные
            
            if(word[0].isupper()):
                if(word.isupper()):
                    labels[index] = 2
                else:
                    labels[index] = 1
            else:
                labels[index] = 0
            index += 1
            
    if(not test):    
        return list(features), list(labels)
    else:
        return list(features)

In [6]:
from collections import Counter

# Подсчитаны все символы в верхнем регистре в трейн сете (из Analysis.ipynb)

counter = Counter({'О': 35312904, ' ': 25422815, 'Н': 24537989, 
                   'Е': 23969940, 'Т': 21420192, 'С': 19892386, 
                   'А': 16809787, 'И': 14950389, 'Р': 13874547, 
                   'В': 13280775, '"': 9695019, 'К': 7858626, 
                   'Г': 5914467, 'Л': 5851872, 'Б': 5428314, 
                   'Й': 5352151, '\n': 5232476, 'Ь': 4866345, 
                   'П': 4642709, 'Д': 4606840, 'М': 4369257, 
                   'Ч': 3898716, 'Я': 3613597, 'У': 3592997, 
                   'Щ': 3302510, 'Ю': 3018424, 'З': 2625035, 
                   'Ц': 2040518, 'Ы': 1500303, '-': 1443125, 
                   'Ж': 1273203, 'Ф': 1136416, 'Х': 1122008, 
                   'Ш': 674331, 'Э': 502713, '.': 406987, 
                   '№': 214440, '1': 194707, '2': 167236, 
                   '(': 124233, ')': 124117, '0': 102588, 
                   ',': 95619, '3': 92674, '4': 73813, 
                   '5': 68067, '7': 58772, '9': 58326, 
                   '6': 56049, 'Ъ': 55448, '8': 47595, 
                   'Ё': 29626, 'I': 29581, '/': 28983, '+': 25175, 
                   'X': 13675, "'": 12332, '`': 8071, 'N': 7230, 
                   'T': 5980, 'V': 5571, 'O': 5074, 'A': 5020, 
                   'E': 4842, 'R': 4648, 'L': 4577, 'S': 4524, 
                   'C': 4156, 'P': 3837, 'D': 3460, 'M': 2589, 
                   ':': 1590, '!': 1578, '&': 1530, 'U': 1491, 
                   'G': 1320, '<': 1152, 'H': 1102, '>': 1069, 
                   'F': 1058, 'Y': 1054, 'B': 1005, 'K': 927, 
                   ';': 892, '\\': 634, 'W': 541, '°': 509, 
                   '«': 427, 'Z': 379, '»': 378, '*': 374, '%': 237, 
                   '_': 237, 'J': 226, '?': 120, 'Q': 58,
                   '=': 48, '@': 45, '–': 41, '^': 29, 
                   '{': 24, '}': 23, '[': 16, '~': 16, ']': 14, 
                   '$': 12, '#': 12, '‹': 10, '›': 10, '·': 8, 
                   '|': 7, '“': 6, '”': 6, '©': 3, '—': 2, 
                   '§': 2, '’': 2, '™': 2, '®': 1, 'Є': 1, 
                   '\xa0': 1, '±': 1, '…': 1, '¶': 1})

print('Число уникальных символов :', len(counter))
encoding = {}
i = 0
for char in counter:
    encoding[char] = i/len(counter)
    i += 1

Число уникальных символов : 124


<font size='4'>Функция создания датасета</font>

In [7]:
def generate_dataset(companies, test=False):
    features = np.array([])
    labels = np.array([])
    for company in companies:
        curr_features, curr_labels = extract_features(company)
        if(len(features)):
            features += curr_features
            labels += curr_labels
        else:
            features = curr_features
            labels = curr_labels
    if(not test):    
        return features, labels
    else:
        return features 

<font size='4'>Функция для преобразования названия компании</font>

In [8]:
def transform(company, labels):
    word_index = 0
    corrected = tokenize(company, spaces=True)
    for i, word in enumerate(corrected):
        if (has_letters(word)):
            if (labels[word_index] == 0):
                corrected[i] = word.lower()
            elif(labels[word_index] == 1):
                corrected[i] = word[0] + word[1:].lower()
            word_index += 1
    return ''.join(corrected)

<font size='4'>Функция для подсчёта метрики</font>

In [9]:
from sklearn.metrics import f1_score

def score(true_name, pred_name):
    length = len(true_name) 
    true_register = np.zeros(length)
    pred_register = np.zeros(length)
    for i in range(length):
        true_register[i] = int(true_name[i].isupper())
        pred_register[i] = int(pred_name[i].isupper())
    return f1_score(true_register, pred_register)

8 гигабайт оперативной памяти хватило на 200000 обработанных компаний

In [10]:
sample_dataset = random.sample(train_companies, 200000)

train_dataset = sample_dataset[:180000]
test_dataset = sample_dataset[180000:]

X_train, y_train = generate_dataset(train_dataset)

In [11]:
len(X_train)

1080895

<font size='4'>Обучения самой модели и немного тюнинга гиперпараметров</font>

In [12]:
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier

param_grid = {
    'learning_rate': [0.01, 0.05, 0.1]
}

clf = CatBoostClassifier(task_type='GPU', random_seed=42)

gs = GridSearchCV(clf, param_grid, cv=5)

gs.fit(X_train, y_train)

0:	learn: 1.0857305	total: 46.7ms	remaining: 46.6s
1:	learn: 1.0733687	total: 87.7ms	remaining: 43.7s
2:	learn: 1.0611753	total: 130ms	remaining: 43.2s
3:	learn: 1.0493517	total: 171ms	remaining: 42.7s
4:	learn: 1.0378457	total: 212ms	remaining: 42.1s
5:	learn: 1.0266324	total: 252ms	remaining: 41.8s
6:	learn: 1.0156997	total: 297ms	remaining: 42.1s
7:	learn: 1.0051531	total: 335ms	remaining: 41.6s
8:	learn: 0.9947451	total: 374ms	remaining: 41.2s
9:	learn: 0.9845899	total: 414ms	remaining: 40.9s
10:	learn: 0.9747831	total: 451ms	remaining: 40.6s
11:	learn: 0.9650749	total: 493ms	remaining: 40.6s
12:	learn: 0.9556077	total: 533ms	remaining: 40.4s
13:	learn: 0.9463606	total: 572ms	remaining: 40.3s
14:	learn: 0.9372405	total: 611ms	remaining: 40.1s
15:	learn: 0.9284238	total: 650ms	remaining: 40s
16:	learn: 0.9197329	total: 690ms	remaining: 39.9s
17:	learn: 0.9112582	total: 734ms	remaining: 40.1s
18:	learn: 0.9029695	total: 775ms	remaining: 40s
19:	learn: 0.8949151	total: 815ms	remaining

161:	learn: 0.4461605	total: 6.33s	remaining: 32.8s
162:	learn: 0.4450271	total: 6.37s	remaining: 32.7s
163:	learn: 0.4439044	total: 6.41s	remaining: 32.7s
164:	learn: 0.4428262	total: 6.45s	remaining: 32.6s
165:	learn: 0.4417338	total: 6.49s	remaining: 32.6s
166:	learn: 0.4406732	total: 6.52s	remaining: 32.5s
167:	learn: 0.4395969	total: 6.56s	remaining: 32.5s
168:	learn: 0.4385273	total: 6.6s	remaining: 32.5s
169:	learn: 0.4375056	total: 6.64s	remaining: 32.4s
170:	learn: 0.4365093	total: 6.67s	remaining: 32.4s
171:	learn: 0.4355054	total: 6.71s	remaining: 32.3s
172:	learn: 0.4345550	total: 6.75s	remaining: 32.3s
173:	learn: 0.4335733	total: 6.79s	remaining: 32.2s
174:	learn: 0.4325788	total: 6.82s	remaining: 32.2s
175:	learn: 0.4316365	total: 6.86s	remaining: 32.1s
176:	learn: 0.4306506	total: 6.89s	remaining: 32.1s
177:	learn: 0.4297145	total: 6.93s	remaining: 32s
178:	learn: 0.4287510	total: 6.97s	remaining: 32s
179:	learn: 0.4278312	total: 7s	remaining: 31.9s
180:	learn: 0.426919

323:	learn: 0.3524135	total: 12.3s	remaining: 25.6s
324:	learn: 0.3521720	total: 12.3s	remaining: 25.6s
325:	learn: 0.3519413	total: 12.3s	remaining: 25.5s
326:	learn: 0.3517225	total: 12.4s	remaining: 25.5s
327:	learn: 0.3514368	total: 12.4s	remaining: 25.4s
328:	learn: 0.3512030	total: 12.4s	remaining: 25.4s
329:	learn: 0.3509477	total: 12.5s	remaining: 25.3s
330:	learn: 0.3506908	total: 12.5s	remaining: 25.3s
331:	learn: 0.3504733	total: 12.5s	remaining: 25.2s
332:	learn: 0.3502543	total: 12.6s	remaining: 25.2s
333:	learn: 0.3500301	total: 12.6s	remaining: 25.1s
334:	learn: 0.3497453	total: 12.6s	remaining: 25.1s
335:	learn: 0.3494689	total: 12.7s	remaining: 25s
336:	learn: 0.3492325	total: 12.7s	remaining: 25s
337:	learn: 0.3490150	total: 12.7s	remaining: 24.9s
338:	learn: 0.3487619	total: 12.8s	remaining: 24.9s
339:	learn: 0.3485199	total: 12.8s	remaining: 24.8s
340:	learn: 0.3482435	total: 12.8s	remaining: 24.8s
341:	learn: 0.3479311	total: 12.9s	remaining: 24.8s
342:	learn: 0.34

487:	learn: 0.3215629	total: 17.8s	remaining: 18.7s
488:	learn: 0.3214513	total: 17.8s	remaining: 18.6s
489:	learn: 0.3212989	total: 17.9s	remaining: 18.6s
490:	learn: 0.3211759	total: 17.9s	remaining: 18.6s
491:	learn: 0.3210311	total: 17.9s	remaining: 18.5s
492:	learn: 0.3209240	total: 18s	remaining: 18.5s
493:	learn: 0.3208340	total: 18s	remaining: 18.4s
494:	learn: 0.3207327	total: 18s	remaining: 18.4s
495:	learn: 0.3205972	total: 18.1s	remaining: 18.3s
496:	learn: 0.3204633	total: 18.1s	remaining: 18.3s
497:	learn: 0.3203117	total: 18.1s	remaining: 18.3s
498:	learn: 0.3201699	total: 18.2s	remaining: 18.2s
499:	learn: 0.3200155	total: 18.2s	remaining: 18.2s
500:	learn: 0.3198906	total: 18.2s	remaining: 18.1s
501:	learn: 0.3197523	total: 18.3s	remaining: 18.1s
502:	learn: 0.3196362	total: 18.3s	remaining: 18.1s
503:	learn: 0.3195043	total: 18.3s	remaining: 18s
504:	learn: 0.3193476	total: 18.4s	remaining: 18s
505:	learn: 0.3192119	total: 18.4s	remaining: 18s
506:	learn: 0.3190806	to

647:	learn: 0.3052192	total: 22.9s	remaining: 12.4s
648:	learn: 0.3051553	total: 22.9s	remaining: 12.4s
649:	learn: 0.3050614	total: 22.9s	remaining: 12.3s
650:	learn: 0.3049649	total: 23s	remaining: 12.3s
651:	learn: 0.3048743	total: 23s	remaining: 12.3s
652:	learn: 0.3047928	total: 23s	remaining: 12.2s
653:	learn: 0.3047263	total: 23.1s	remaining: 12.2s
654:	learn: 0.3046337	total: 23.1s	remaining: 12.2s
655:	learn: 0.3045839	total: 23.1s	remaining: 12.1s
656:	learn: 0.3045081	total: 23.2s	remaining: 12.1s
657:	learn: 0.3044414	total: 23.2s	remaining: 12.1s
658:	learn: 0.3043422	total: 23.2s	remaining: 12s
659:	learn: 0.3042329	total: 23.3s	remaining: 12s
660:	learn: 0.3041601	total: 23.3s	remaining: 11.9s
661:	learn: 0.3041002	total: 23.3s	remaining: 11.9s
662:	learn: 0.3040403	total: 23.3s	remaining: 11.9s
663:	learn: 0.3039374	total: 23.4s	remaining: 11.8s
664:	learn: 0.3038538	total: 23.4s	remaining: 11.8s
665:	learn: 0.3037470	total: 23.5s	remaining: 11.8s
666:	learn: 0.3036701	

...

394:	learn: 0.2307929	total: 15.5s	remaining: 23.8s
395:	learn: 0.2307589	total: 15.6s	remaining: 23.8s
396:	learn: 0.2306673	total: 15.6s	remaining: 23.7s
397:	learn: 0.2306146	total: 15.7s	remaining: 23.7s
398:	learn: 0.2305604	total: 15.7s	remaining: 23.6s
399:	learn: 0.2305241	total: 15.7s	remaining: 23.6s
400:	learn: 0.2304197	total: 15.8s	remaining: 23.6s
401:	learn: 0.2303748	total: 15.8s	remaining: 23.5s
402:	learn: 0.2303258	total: 15.8s	remaining: 23.5s
403:	learn: 0.2302161	total: 15.9s	remaining: 23.4s
404:	learn: 0.2301378	total: 15.9s	remaining: 23.4s
405:	learn: 0.2300678	total: 16s	remaining: 23.4s
406:	learn: 0.2300336	total: 16s	remaining: 23.3s
407:	learn: 0.2299956	total: 16s	remaining: 23.3s
408:	learn: 0.2299571	total: 16.1s	remaining: 23.2s
409:	learn: 0.2298652	total: 16.1s	remaining: 23.2s
410:	learn: 0.2297978	total: 16.2s	remaining: 23.2s
411:	learn: 0.2297408	total: 16.2s	remaining: 23.1s
412:	learn: 0.2296642	total: 16.2s	remaining: 23.1s
413:	learn: 0.2295

558:	learn: 0.2216143	total: 21.9s	remaining: 17.2s
559:	learn: 0.2215704	total: 21.9s	remaining: 17.2s
560:	learn: 0.2215296	total: 21.9s	remaining: 17.2s
561:	learn: 0.2215144	total: 22s	remaining: 17.1s
562:	learn: 0.2214539	total: 22s	remaining: 17.1s
563:	learn: 0.2214118	total: 22s	remaining: 17s
564:	learn: 0.2213711	total: 22.1s	remaining: 17s
565:	learn: 0.2212819	total: 22.1s	remaining: 17s
566:	learn: 0.2212222	total: 22.2s	remaining: 16.9s
567:	learn: 0.2211908	total: 22.2s	remaining: 16.9s
568:	learn: 0.2211628	total: 22.2s	remaining: 16.8s
569:	learn: 0.2211452	total: 22.3s	remaining: 16.8s
570:	learn: 0.2211179	total: 22.3s	remaining: 16.8s
571:	learn: 0.2210858	total: 22.3s	remaining: 16.7s
572:	learn: 0.2210647	total: 22.4s	remaining: 16.7s
573:	learn: 0.2209808	total: 22.4s	remaining: 16.6s
574:	learn: 0.2209197	total: 22.5s	remaining: 16.6s
575:	learn: 0.2209008	total: 22.5s	remaining: 16.6s
576:	learn: 0.2208779	total: 22.5s	remaining: 16.5s
577:	learn: 0.2208351	to

718:	learn: 0.2164483	total: 27.8s	remaining: 10.9s
719:	learn: 0.2164142	total: 27.8s	remaining: 10.8s
720:	learn: 0.2163772	total: 27.9s	remaining: 10.8s
721:	learn: 0.2163506	total: 27.9s	remaining: 10.7s
722:	learn: 0.2163330	total: 27.9s	remaining: 10.7s
723:	learn: 0.2162841	total: 28s	remaining: 10.7s
724:	learn: 0.2162520	total: 28s	remaining: 10.6s
725:	learn: 0.2162117	total: 28.1s	remaining: 10.6s
726:	learn: 0.2161866	total: 28.1s	remaining: 10.6s
727:	learn: 0.2161558	total: 28.1s	remaining: 10.5s
728:	learn: 0.2161357	total: 28.2s	remaining: 10.5s
729:	learn: 0.2161265	total: 28.2s	remaining: 10.4s
730:	learn: 0.2161093	total: 28.3s	remaining: 10.4s
731:	learn: 0.2160975	total: 28.3s	remaining: 10.4s
732:	learn: 0.2160701	total: 28.3s	remaining: 10.3s
733:	learn: 0.2160394	total: 28.4s	remaining: 10.3s
734:	learn: 0.2160347	total: 28.4s	remaining: 10.2s
735:	learn: 0.2160036	total: 28.4s	remaining: 10.2s
736:	learn: 0.2159839	total: 28.5s	remaining: 10.2s
737:	learn: 0.21

882:	learn: 0.2123339	total: 34s	remaining: 4.51s
883:	learn: 0.2123149	total: 34.1s	remaining: 4.47s
884:	learn: 0.2122858	total: 34.1s	remaining: 4.43s
885:	learn: 0.2122695	total: 34.2s	remaining: 4.39s
886:	learn: 0.2122550	total: 34.2s	remaining: 4.36s
887:	learn: 0.2122378	total: 34.2s	remaining: 4.32s
888:	learn: 0.2122343	total: 34.3s	remaining: 4.28s
889:	learn: 0.2122219	total: 34.3s	remaining: 4.24s
890:	learn: 0.2122038	total: 34.3s	remaining: 4.2s
891:	learn: 0.2121904	total: 34.4s	remaining: 4.16s
892:	learn: 0.2121655	total: 34.4s	remaining: 4.12s
893:	learn: 0.2121556	total: 34.4s	remaining: 4.08s
894:	learn: 0.2121387	total: 34.5s	remaining: 4.05s
895:	learn: 0.2121142	total: 34.5s	remaining: 4.01s
896:	learn: 0.2121018	total: 34.6s	remaining: 3.97s
897:	learn: 0.2120808	total: 34.6s	remaining: 3.93s
898:	learn: 0.2120624	total: 34.6s	remaining: 3.89s
899:	learn: 0.2120479	total: 34.7s	remaining: 3.85s
900:	learn: 0.2120380	total: 34.7s	remaining: 3.81s
901:	learn: 0.2

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f8ef4ebb9d0>,
             param_grid={'learning_rate': [0.01, 0.05, 0.1]})

In [13]:
gs.best_params_

{'learning_rate': 0.1}

<font size='4'>Проверяем, чему научилась модель</font>

In [14]:
predictions = []
for company in test_dataset:
    features = extract_features(company, test=True)
    labels = gs.predict(features)
    predictions.append(transform(company.upper(), labels))
    
print(score(' '.join(test_dataset), ' '.join(predictions)))

0.81660989861093


In [15]:
test_dataset[:5]

['Жилищный кооператив "Поселковая-4"',
 'Общество с ограниченной ответственностью "Кубик групп"',
 'Общество с ограниченной ответственностью "МКСмедиа"',
 'Общество с ограниченной ответственностью "Сантехкомплект-ОПТ"',
 'муниципальное казенное учреждение культуры "Культурно-досуговое объединение Хуторского сельсовета"']

In [16]:
predictions[:5]

['Жилищный кооператив "Поселковая-4"',
 'Общество с ограниченной ответственностью "Кубик Групп"',
 'Общество с ограниченной ответственностью "Мксмедиа"',
 'Общество с ограниченной ответственностью "Сантехкомплект-ОПТ"',
 'Муниципальное казенное учреждение культуры "Культурно-досуговое объединение Хуторского сельсовета"']

In [17]:
result = open('result.txt', 'w')
for company in test_companies:
    result.write(transform(company, gs.predict(extract_features(company, test=True))) + '\n')

# Результаты
<font size=4>Деревья быстро обучаются (17 минут на моём ноутбуке) и быстро обрабатывают тест датасет (около 15 минут), при этом выдавая неплохой f1-score.</font>